In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os
import numpy as np
from keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Embedding, Dropout, GlobalAveragePooling1D, Flatten, SpatialDropout1D, Bidirectional
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/dga_dataset/dataset_num_balanced.csv')

In [ ]:
print(data)

          domTLD  domSLD  Ldom  LTLD  LSLD  LOLD  Ddom  HwP  HIP  LCc  ...  \
0        3082872     357  13.0  10.0   2.0   0.0   2.0  0.0  0.0  2.0  ...   
1         308238     153  14.0  10.0   3.0   0.0   2.0  0.0  0.0  2.0  ...   
2        2696104     153  15.0  11.0   3.0   0.0   2.0  0.0  0.0  2.0  ...   
3        1068629     153  19.0  15.0   3.0   0.0   2.0  0.0  0.0  4.0  ...   
4         427867     578  13.0  10.0   2.0   0.0   2.0  0.0  0.0  2.0  ...   
...          ...     ...   ...   ...   ...   ...   ...  ...  ...  ...  ...   
1887161  2906936     153  14.0  10.0   3.0   0.0   2.0  0.0  0.0  5.0  ...   
1887162  2584431     153  28.0  24.0   3.0   0.0   2.0  0.0  0.0  4.0  ...   
1887163  2145418     579  20.0  17.0   2.0   0.0   2.0  0.0  0.0  2.0  ...   
1887164  3550279     153  30.0  26.0   3.0   0.0   2.0  0.0  0.0  3.0  ...   
1887165  3396447     153  24.0  20.0   3.0   0.0   2.0  0.0  0.0  6.0  ...   

           RAv    RAl  RAn    RAs     Rc     Rv     Rl   Rn    

In [ ]:
y = data[['label']].values
del data['label']
del data['HIP']

In [ ]:
# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Fit the scaler to your dataset
scaler.fit(data)

# Transform your dataset using the scaler
df_norm = scaler.transform(data)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_norm, y,stratify=y, test_size=0.2, random_state=42)

In [ ]:
x_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))

In [ ]:
x_train.shape

(1509732, 35, 1)

In [ ]:
model = Sequential()
# input layer and LSTM layer with 50 neurons
model.add(LSTM(units=128, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units=128, return_sequences=True))
model.add(Dropout(0,2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
# training the model on training dataset
early_stop = EarlyStopping(monitor='val_loss', patience=2)
history = model.fit(x_train, y_train, epochs=100, batch_size=5000,callbacks =[early_stop],validation_split=0.2)

Epoch 1/100
242/242 [==============================] - 2031s 8s/step - loss: 0.6492 - accuracy: 0.6116 - val_loss: 0.6293 - val_accuracy: 0.6337
Epoch 2/100
242/242 [==============================] - 2038s 8s/step - loss: 0.6115 - accuracy: 0.6500 - val_loss: 0.5861 - val_accuracy: 0.6708
Epoch 3/100
242/242 [==============================] - 2048s 8s/step - loss: 0.5774 - accuracy: 0.6782 - val_loss: 0.5558 - val_accuracy: 0.6979
Epoch 4/100
242/242 [==============================] - 1996s 8s/step - loss: 0.5505 - accuracy: 0.7015 - val_loss: 0.5444 - val_accuracy: 0.7076
Epoch 5/100
242/242 [==============================] - 2022s 8s/step - loss: 0.5341 - accuracy: 0.7148 - val_loss: 0.5201 - val_accuracy: 0.7242
Epoch 6/100
242/242 [==============================] - 2032s 8s/step - loss: 0.5215 - accuracy: 0.7236 - val_loss: 0.5009 - val_accuracy: 0.7382
Epoch 7/100
242/242 [==============================] - 2026s 8s/step - loss: 0.5035 - accuracy: 0.7365 - val_loss: 0.4947 - val_ac

In [ ]:
x_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

# predicting target attribute on testing dataset
test_results = model.evaluate(x_test, y_test, verbose=1)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]*100}%')

11795/11795 [==============================] - 74s 6ms/step - loss: 0.3707 - accuracy: 0.8145
Test results - Loss: 0.37069424986839294 - Accuracy: 81.4464271068573%


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# ...

# predicting target attribute on testing dataset
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).flatten()

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy * 100}%")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")